Trainer for revised model

In [13]:
class opt():
    hparamDatasetPath = "/Users/nikolai/Downloads/UPC/VSC/Project/dataset/sequences"
    hparamYamlConfigPath = "/Users/nikolai/Downloads/UPC/VSC/Project/Workspace-AD-LiDAR/F0_Visualization/semantic-kitti-api/config/semantic-kitti.yaml"
    hparamNumPoints = 4000
    hparamNumberOfClasses = 20
    #hparamClassChoice = 'bus'
    hparamDatasetSequence = '04'
    hparamTrainBatchSize = 32
    hparamTrainNumEpochs = 100
    hparamOutputFolder = "/Users/nikolai/Downloads/UPC/VSC/Project/output/"
    hparamDeviceType = 'cpu'
    hparamFeatureTransform = False

In [17]:
from __future__ import print_function
import argparse
import os
import random
import torch
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
#from A0_Configuration.hyperparam import opt
from B0_Dataset.dataset import SemanticKittiDataset
from D0_Modeling.model import SegmentationPointNet
from B1_Dataloader.dataloader import DataLoader_
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np

blue = lambda x: '\033[94m' + x + '\033[0m'

torch.manual_seed(123)

#torch.utils.data.random_split(SemanticKittiDataset, [0.2, 0.8], generator=torch.Generator().manual_seed(42))
#train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(SemanticKittiDataset, [0.7, 0.2, 0.1])

train_dataset = SemanticKittiDataset(
    dst_hparamDatasetPath=opt.hparamDatasetPath[0],
    dst_hparamDatasetSequence=opt.hparamDatasetSequence,
    dst_hparamYamlConfigPath=opt.hparamYamlConfigPath[0],
    dst_hparamNumberOfRandomPoints=opt.hparamNumPoints,
    dst_hparamActionType='train',
    dst_hparamPointDimension=4)
y = next(iter(train_dataset))

val_dataset = SemanticKittiDataset(
    dst_hparamDatasetPath=opt.hparamDatasetPath[0],
    dst_hparamDatasetSequence=opt.hparamDatasetSequence,
    dst_hparamYamlConfigPath=opt.hparamYamlConfigPath[0],
    dst_hparamNumberOfRandomPoints=opt.hparamNumPoints,
    dst_hparamActionType='val',
    dst_hparamPointDimension=4)

test_dataset = SemanticKittiDataset(
    dst_hparamDatasetPath=opt.hparamDatasetPath[0],
    dst_hparamDatasetSequence=opt.hparamDatasetSequence,
    dst_hparamYamlConfigPath=opt.hparamYamlConfigPath[0],
    dst_hparamNumberOfRandomPoints=opt.hparamNumPoints,
    dst_hparamActionType='test')

train_dataloader = DataLoader_(
    dataset = train_dataset,
    batch_size=opt.hparamTrainBatchSize,
    shuffle=True)

val_dataloader = DataLoader_(
    dataset = val_dataset,
    batch_size=opt.hparamBatchSize,
    shuffle=True)

test_dataloader = DataLoader_(
    dataset = test_dataset,
    batch_size=opt.hparamBatchSize,
    shuffle=True)

torch.utils.data.random_split(SemanticKittiDataset, [2, 8], generator=torch.Generator().manual_seed(42))

print(len(train_dataset), len(val_dataset), len(test_dataset))
print('classes', opt.hparamNumberOfClasses)

#try:
#    os.makedirs(opt.hparamOutputFolder)
#except OSError:
#    pass

TypeError: object of type 'type' has no len()

In [36]:
num_classes=opt.hparamNumberOfClasses
feature_transform=opt.hparamFeatureTransform
model = SegmentationPointNet(num_classes, feature_transform)

torch.cuda.is_available()
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
model = model.to(opt.hparamDeviceType)

num_batch = len(train_dataset) / opt.hparamBatchSize

for epoch in range(opt.hparamNumberOfEpochs):
    scheduler.step()
    for i, data in enumerate(train_dataloader, 0):
        points, target = data
        points = points.transpose(2, 1)
        points, target = points.to(opt.hparamDeviceType), target.to(opt.hparamDeviceType)
        optimizer.zero_grad()
        model = model.train()
        pred,trans_feat = model(points) # pred.shape=torch.Size([32, 4000, 20]) & target.shape = torch.Size([32, 4000])
        pred = pred.view(-1, num_classes) 
        target = target.view(-1, 1)[:, 0]
        #print(pred.size(), target.size())
        loss = F.nll_loss(pred, target) #Consecutive_Predictions(Target) pred=[4000*32, 20])
        if opt.hparamFeatureTransform:
            loss += feature_transform(trans_feat) * 0.001
        loss.backward()
        optimizer.step()
        pred_choice = pred.data.max(1)[1]
        correct = pred_choice.eq(target.data).cpu().sum()
        print('[%d: %d/%d] train loss: %f accuracy: %f' % (epoch, i, num_batch, loss.item(), correct.item()/float(opt.hparamBatchSize * 2500)))

        if i % 10 == 0:
            j, data = next(enumerate(test_dataloader, 0))
            points, target = data
            points = points.transpose(2, 1)
            points, target = points.to(opt.hparamDeviceType), target.to(opt.hparamDeviceType)
            model = model.eval()
            pred,_ = model(points)
            pred = pred.view(-1, num_classes)
            target = target.view(-1, 1)[:, 0]
            loss = F.nll_loss(pred, target)
            pred_choice = pred.data.max(1)[1]
            correct = pred_choice.eq(target.data).cpu().sum()
            print('[%d: %d/%d] %s loss: %f accuracy: %f' % (epoch, i, num_batch, blue('test'), loss.item(), correct.item()/float(opt.hparamBatchSize * 2500)))

    torch.save(model.state_dict(), '%s/seg_model_%s_%d.pth' % (opt.hparamOutputFolder, epoch))

/Users/nikolai/opt/miniconda3/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


torch.Size([32, 1024])
torch.Size([32, 9])
torch.Size([32, 3, 3])


IndexError: index 3 is out of bounds for dimension 1 with size 3